In [12]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
# !conda install -c conda-forge geopy --yes 
import geopy.geocoders # convert an address into latitude and longitude values

In [14]:
# !conda ins?tall -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries are imported.')

Libraries are imported.


usage: conda [-h] [-V] command ...
conda: error: argument command: invalid choice: 'ins?tall' (choose from 'clean', 'config', 'create', 'help', 'info', 'install', 'list', 'package', 'remove', 'uninstall', 'search', 'update', 'upgrade')


In [15]:
# Loading the dataset which is about postal codes in Toronto
# This dataset was created in week 3. 
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head()

,Unnamed: 0,Post Code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [17]:
# df_toronto['Borough'] == 'Scarborough'

# selecting only neighborhoods regarding to "Scarborough" borough.
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [19]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
        print("url", url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [20]:

CLIENT_ID = 'MRSTVYMQOEM1B03VCOLOTSTXQQQKWKTJC30LDHLNKJYZ44BH' # your Foursquare ID
CLIENT_SECRET = 'ZISYTO151NTB2MNSNSPTJVD1JIH3ELF0G0QGCWW2XULVNVMF' # your Foursquare Secret
VERSION = '20190623' # Foursquare API version

In [21]:
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Post Code']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
url https://api.foursquare.com/v2/venues/explore?&client_id=MRSTVYMQOEM1B03VCOLOTSTXQQQKWKTJC30LDHLNKJYZ44BH&client_secret=ZISYTO151NTB2MNSNSPTJVD1JIH3ELF0G0QGCWW2XULVNVMF&v=20190623&ll=43.8066863,-79.19435340000003&radius=1000&limit=500
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
url https://api.foursquare.com/v2/venues/explore?&client_id=MRSTVYMQOEM1B03VCOLOTSTXQQQKWKTJC30LDHLNKJYZ44BH&client_secret=ZISYTO151NTB2MNSNSPTJVD1JIH3ELF0G0QGCWW2XULVNVMF&v=20190623&ll=43.7845351,-79.16049709999999&radius=1000&limit=500
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
url https://api.foursquare.com/v2/venues/explore?&client_id=MRSTVYMQOEM1B03VCOLOTSTXQQQKWKTJC30LDHLNKJYZ44BH&client_secret=ZISYTO151NTB2MNSNSPTJVD1JIH3ELF0G0QGCWW2XULVNVMF&v=20190623&ll=43.7635726,-79.1887115&radius=1000&limit=500
3.
Data is Obtained, for th

In [22]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')

Received Data from Internet is Saved to Computer.


In [23]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)

print(type(Scarborough_foursquare_dataset))
Scarborough_foursquare_dataset

<class 'list'>


[{'Postal Code': 'M1B',
  'Neighborhood(s)': 'Rouge, Malvern',
  'Latitude': 43.8066863,
  'Longitude': -79.19435340000003,
  'Crawling_result': [{'reasons': {'count': 0,
     'items': [{'summary': 'This spot is popular',
       'type': 'general',
       'reasonName': 'globalInteractionReason'}]},
    'venue': {'id': '4d669cba83865481c948fa53',
     'name': 'Images Salon & Spa',
     'location': {'address': '8130 Sheppard Ave E',
      'crossStreet': 'Morningside Ave',
      'lat': 43.80228301948931,
      'lng': -79.19856472801668,
      'labeledLatLngs': [{'label': 'display',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668}],
      'distance': 595,
      'postalCode': 'M1B 3W3',
      'cc': 'CA',
      'city': 'Toronto',
      'state': 'ON',
      'country': 'Canada',
      'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
       'Toronto ON M1B 3W3',
       'Canada']},
     'categories': [{'id': '4bf58dd8d48988d1ed941735',
       'name': 'Spa',
      

In [24]:
def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [25]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Rouge, Malvern" Negihborhood(s) is:
18
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is:
23
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
8
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
30
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1K" Posal Code and "East Birchmount Park, Ionview, Kennedy Park" Negihborhood(s) is:
25
Number of Venuse in Coordination "M1L" Posal Code and "Clairlea, Golden Mile, Oakridge" Negihborhood(s) is:
29
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside, Scarborough Village West" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1N" Pos

In [26]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,Spa,595
1,M1B,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,This spot is popular,Caribbean Restaurant,912
2,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,Paper / Office Supplies Store,735
3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
4,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,387


In [27]:
scarborough_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
375,M1W,"L'Amoreaux West, Steeles West",43.799525,-79.318389,L'Amoreaux Scarborough Winter Tennis Club,This spot is popular,Tennis Court,896
376,M1W,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Buddy Cafe,This spot is popular,Chinese Restaurant,973
377,M1W,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Birchwood Plaza,This spot is popular,Shopping Mall,977
378,M1W,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Olympian Swimming,This spot is popular,Gym Pool,978
379,M1W,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Red Sail Boat Bakery 紅帆船西餅麵包,This spot is popular,Bakery,966


In [28]:

scarborough_venues.to_csv('scarborough_venues.csv')

In [29]:
scarborough_venues = pd.read_csv('scarborough_venues.csv')

In [30]:

neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Rouge, Malvern',
 'Highland Creek, Rouge Hill, Port Union',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'East Birchmount Park, Ionview, Kennedy Park',
 'Clairlea, Golden Mile, Oakridge',
 'Cliffcrest, Cliffside, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Maryvale, Wexford',
 'Agincourt',
 "Clarks Corners, Sullivan, Tam O'Shanter",
 "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
 "L'Amoreaux West, Steeles West"]

In [31]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,45,45,45,45,45,45,45
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",24,24,24,24,24,24,24
"Birch Cliff, Cliffside West",15,15,15,15,15,15,15
Cedarbrae,30,30,30,30,30,30,30
"Clairlea, Golden Mile, Oakridge",29,29,29,29,29,29,29


In [32]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 111 uniques categories.
Here is the list of different categories:


['Spa',
 'Caribbean Restaurant',
 'Paper / Office Supplies Store',
 'Fast Food Restaurant',
 'Coffee Shop',
 'Hobby Shop',
 'Bus Station',
 'African Restaurant',
 'Auto Workshop',
 'Chinese Restaurant',
 'Greek Restaurant',
 'Fruit & Vegetable Store',
 'Gym',
 'Bakery',
 'Sandwich Place',
 'Burger Joint',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Park',
 'Fried Chicken Joint',
 'Pizza Place',
 'Liquor Store',
 'Food & Drink Shop',
 'Smoothie Shop',
 'Beer Store',
 'Pharmacy',
 'Discount Store',
 'Sports Bar',
 'Bank',
 'Supermarket',
 'Indian Restaurant',
 'Electronics Store',
 'Hakka Restaurant',
 'Thai Restaurant',
 'Athletics & Sports',
 'Music Store',
 'Wings Joint',
 'Yoga Studio',
 'Lounge',
 'Grocery Store',
 'Asian Restaurant',
 'Rental Car Location',
 'Bus Line',
 'Sporting Goods Shop',
 'Restaurant',
 'Convenience Store',
 'Auto Garage',
 'Train Station',
 'Japanese Restaurant',
 'Bowling Alley',
 'Department Store',
 'Hockey Arena',
 'Metro Station',
 'Light

In [33]:

# Just for fun and deeper understanding
print(type(scarborough_venues[['Venue Category']]))

print(type(scarborough_venues['Venue Category']))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [55]:
# one hot encoding
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hotpot Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1B,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,This spot is popular,912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
# This list is created manually 
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant', 
 'BBQ Joint', 
 'Bakery', 
 'Breakfast Spot',
 'Burger Joint',  
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant', 
 'Diner',
 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store', 
 'Greek Restaurant',
 'Grocery Store', 
 'Hakka Restaurant',
 'Hong Kong Restaurant',
 'Hotpot Restaurant', 
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant', 
 'Mediterranean Restaurant', 
 'Mexican Restaurant',
 'Middle Eastern Restaurant', 
 'Noodle House', 
 'Pizza Place', 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant', 
 'Sushi Restaurant',
 'Taiwanese Restaurant', 
 'Thai Restaurant', 
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

In [57]:
scarborough_onehot = scarborough_onehot.drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.describe()

,Unnamed: 0,Distance,African Restaurant,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hotpot Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
count,16.000000,16.000000,16.0000,16.000000,16.000000,16.0000,16.000000,16.0000,16.0000,16.0000,16.000000,16.000000,16.000000,16.0000,16.000,16.000000,16.000000,16.000000,16.0000,16.000000,16.00000,16.000000,16.0000,16.000000,16.0000,16.000000,16.000000,16.00000,16.0000,16.000000,16.0000,16.000000,16.0000,16.0000,16.000000,16.000000,16.000000,16.000000,16.0000,16.000000,16.0000,16.0000,16.0000,16.000000,16.0000,16.000000,16.0000,16.000000,16.000000,16.000000,16.000000,16.0000,16.000000,16.0000,16.000000,16.0000,16.0000,16.000000,16.000000,16.000000,16.0000,16.000000,16.0000,16.000000,16.0000,16.000000,16.000000,16.0000,16.0000,16.000000,16.000000,16.00,16.0000,16.0000,16.000000,16.0000,16.0000,16.000000,16.0000,16.000000,16.000000,16.0000,16.0000,16.0000,16.0000,16.0000,16.000000,16.000000,16.000000,16.000000,16.0000,16.0000,16.000000,16.0000,16.0000,16.000000,16.000000,16.0000,16.000000,16.0000,16.00000,16.0000,16.0000,16.0000,16.000000,16.0000,16.0000,16.0000,16.0000,16.0000,16.000000,16.000000,16.0000
mean,4500.625000,16839.125000,0.0625,0.187500,0.375000,0.0625,0.125000,0.0625,0.0625,0.0625,0.125000,1.000000,0.562500,0.0625,0.125,0.187500,0.125000,0.312500,0.0625,0.562500,0.25000,0.250000,0.0625,0.125000,0.0625,0.125000,0.375000,1.31250,0.0625,1.750000,0.0625,0.187500,0.0625,0.0625,0.125000,0.125000,0.375000,0.312500,0.0625,1.500000,0.0625,0.0625,0.0625,0.250000,0.0625,0.187500,0.0625,0.125000,0.187500,0.625000,0.250000,0.0625,0.125000,0.0625,0.187500,0.0625,0.0625,0.437500,0.437500,0.187500,0.0625,0.125000,0.0625,0.125000,0.0625,0.125000,0.125000,0.0625,0.0625,0.125000,0.125000,0.25,0.0625,0.0625,0.187500,0.0625,0.0625,0.687500,0.0625,0.750000,1.375000,0.0625,0.0625,0.0625,0.0625,0.0625,0.125000,0.312500,0.687500,0.187500,0.0625,0.0625,0.375000,0.0625,0.0625,0.125000,0.125000,0.0625,0.125000,0.0625,0.25000,0.0625,0.0625,0.0625,0.187500,0.0625,0.0625,0.0625,0.0625,0.0625,0.250000,0.187500,0.0625
std,4165.918768,9137.749861,0.2500,0.403113,0.619139,0.2500,0.341565,0.2500,0.2500,0.2500,0.341565,1.032796,0.512348,0.2500,0.500,0.403113,0.341565,0.478714,0.2500,0.727438,0.57735,0.447214,0.2500,0.341565,0.2500,0.341565,0.619139,1.74045,0.2500,1.290994,0.2500,0.403113,0.2500,0.2500,0.341565,0.341565,0.806226,1.014479,0.2500,0.966092,0.2500,0.2500,0.2500,0.447214,0.2500,0.543906,0.2500,0.341

In [58]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

In [59]:
scarborough_onehot

,Unnamed: 0,Distance,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fish Market,Flea Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hockey Arena,Intersection,Light Rail Station,Liquor Store,Lounge,Market,Metro Station,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Sandwich Place,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Tennis Court,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Yoga Studio,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,12375,36397,0,0,0,0,1,2,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,2,0,0,1,1,0,0,2,0,4,0,0,0,0,0,0,2,0,0,0,0,0,0,20,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",8244,17040,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1
"Birch Cliff, Cliffside West",2550,10181,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0
Cedarbrae,2055,18626,1,0,0,0,0,3,1,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,8,3
"Clairlea, Golden Mile, Oakridge",3915,19768,0,0,0,0,0,2,1,0,0,1,0,0,0,2,1,0,0,0,3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,2,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,3,0
"Clarks Corners, Sullivan, Tam O'Shanter",10693,24416,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,2,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,2,2,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,12,1
"Cliffcrest, Cliffside, Scarborough Village West",2028,10366,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",9000,34729,0,0,0,1,0,2,1,0,0,1,1,0,0,0,1,0,0,1,4,0,0,0,0,0,0,0,4,0,0,0,0,0,2,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,4
"East Birchmount Park, Ionview, Kennedy Park",2700,17091,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,4,0,1,0,1,0,0,3,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1


In [60]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

In [61]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Unnamed: 0,Distance,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fish Market,Flea Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hockey Arena,Intersection,Light Rail Station,Liquor Store,Lounge,Market,Metro Station,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Sandwich Place,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Tennis Court,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Yoga Studio,Total Restaurants,Total Joints,Total Sum
G2,10687.5,35563.0,0.0,0.0,0.0,0.5,0.5,2.0,1.0,0.0,0.0,0.5,0.5,0.5,0.5,0.0,0.5,0.0,0.0,0.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,0.5,0.5,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.0,0.0,1.0,0.5,1.0,1.5,0.0,0.5,0.5,0.5,0.0,0.0,1.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.5,2.0,46295.5
G4,8909.0,22482.5,0.0,0.0,0.0,0.0,0.5,1.0,0.5,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,0.0,1.5,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.5,0.0,1.5,0.0,0.0,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,1.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,10.5,1.5,31423.5
G5,8532.0,14390.5,0.0,0.5,0.0,0.0,0.0,2.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.5,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,1.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,8.0,0.5,22946.5
G1,1921.0,16983.0,0.2,0.0,0.2,0.0,0.0,1.2,0.8,0.0,0.0,0.4,0.0,0.2,0.0,0.8,0.4,0.0,0.0,0.0,2.8,0.0,0.4,0.0,0.2,0.0,0.2,0.8,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.8,0.4,0.0,0.0,0.2,0.2,0.6,0.2,0.2,0.2,0.0,0.4,0.0,0.2,0.0,0.0,0.2,0.4,0.0,0.8,1.4,0.0,0.0,0.0,0.0,0.2,0.4,0.8,0.0,0.0,0.0,0.2,0.2,0.4,0.2,0.2,0.2,0.0,0.0,0.2,0.0,0.0,0.2,5.2,1.2,18929.0
G3,1229.6,7927.8,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.4,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.6,0.2,0.2,0.0,0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.2,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,3.2,0.6,9168.0


In [62]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,2
1,"Agincourt North, L'Amoreaux East, Milliken, St...",5
2,"Birch Cliff, Cliffside West",3
3,Cedarbrae,1
4,"Clairlea, Golden Mile, Oakridge",1
5,"Clarks Corners, Sullivan, Tam O'Shanter",4
6,"Cliffcrest, Cliffside, Scarborough Village West",3
7,"Dorset Park, Scarborough Town Centre, Wexford ...",2
8,"East Birchmount Park, Ionview, Kennedy Park",1
9,"Guildwood, Morningside, West Hill",1


In [63]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
5,"Clarks Corners, Sullivan, Tam O'Shanter",4
12,"Maryvale, Wexford",4


In [64]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1T',
 'Neighborhood': "Clarks Corners, Sullivan, Tam O'Shanter",
 'Neighborhood Latitude': 43.7816375,
 'Neighborhood Longitude': -79.3043021}

In [65]:
neigh_summary[neigh_summary['Group'] == 1]

,Neighborhood,Group
3,Cedarbrae,1
4,"Clairlea, Golden Mile, Oakridge",1
8,"East Birchmount Park, Ionview, Kennedy Park",1
9,"Guildwood, Morningside, West Hill",1
13,"Rouge, Malvern",1


In [66]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
0,Agincourt,2
7,"Dorset Park, Scarborough Town Centre, Wexford ...",2


In [67]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1T',
 'Neighborhood': "Clarks Corners, Sullivan, Tam O'Shanter",
 'Neighborhood Latitude': 43.7816375,
 'Neighborhood Longitude': -79.3043021}